In [1]:
import numpy as np
print(np.__version__)

1.25.2


In [2]:
import pandas as pd
print(pd.__version__)

1.5.3


In [3]:
!pip uninstall -y numpy pandas
!pip install -q numpy==1.25.2 pandas==1.5.3 --force-reinstall --no-cache-dir

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 137.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 342.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 353.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 381.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.3 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requir

In [5]:
# For installing the libraries & downloading models from HF Hub
!pip install -q huggingface_hub==0.23.2 \
             tiktoken==0.6.0 \
             pymupdf==1.25.1 \
             langchain==0.1.1 \
             langchain-community==0.0.13 \
             chromadb==0.4.22 \
             sentence-transformers==2.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.1 MB/s

In [8]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85  --force-reinstall --upgrade --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 174.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 276.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 319.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 275.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.2.5 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.3 

In [9]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

import warnings
warnings.filterwarnings('ignore')

# Data Preparation

### Loading the data

In [10]:
# mounting the drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# specify the path for the pdf file

pdf_file = "/content/drive/My Drive/colab_notebooks/RAG_medical_assistant/medical_diagnosis_manual.pdf"

In [12]:
# Initialize the PDF loader using PyMuPDF

pdf_loader = PyMuPDFLoader(pdf_file);

In [13]:
# Load content from the PDF file

text = pdf_loader.load()

# Data Overview

In [ ]:
# Print the content of the third page (index 2)

print(text[2].page_content,end="\n")

Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - Nutritional Disorders    ...............................................................................................................................................................
53
Chapter 1. Nutrition: General Considerations    ......................................................................................................

# Question Answering using LLM

In [14]:
# Download Llama-2-13B-Chat (quantized)

model_path = hf_hub_download(
    repo_id = "TheBloke/Llama-2-13B-chat-GGUF",
    filename = "llama-2-13b-chat.Q5_K_M.gguf"
)

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [15]:
# Initialize the LLaMA language model with specified configuration

llm = Llama(
    model_path=model_path,
    n_threads=2,          # CPU threads for computation
    n_batch=512,          # Tokens processed in parallel
    n_gpu_layers=42,      # Layers offloaded to GPU
    n_ctx=2048,           # Context window size
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [16]:
# function to generate, process, and return the response from the LLM

def generate_llama_response(user_prompt):

    # System message
    system_message = """
    [INST]<<SYS>> Respond to the user question based on the user prompt<</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query_1 = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_llama_response(query_1)
print(response)

 The management of sepsis in a critical care unit (CCU) follows a standardized protocol that includes early recognition, aggressive fluid resuscitation, antibiotics, and close monitoring. Here are the key components of the protocol:

1. Early Recognition: The CCU team should be vigilant in identifying patients at risk for sepsis, using criteria such as SIRS (Systemic Inflammatory Response Syndrome) or QSOFA (Quick Sequential Organ Failure Assessment).
2. Aggressive Fluid Resuscitation: The goal is to achieve a mean arterial pressure (MAP) of 65-70 mmHg, and to maintain a central venous oxygen saturation (ScvO2) of >70%. Crystalloids are the first choice for fluid resuscitation, but vasopressors may be needed if fluid responsiveness is uncertain.
3. Antibiotics: Broad-spectrum antibiotics should be administered as soon as possible, ideally within the first hour of recognition of sepsis. The selection of antibiotics should cover common causes of sepsis, such as Gram-positive and Gram-neg

- The model has answered the question fairly with numerical points.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response = generate_llama_response(query_2)
print(response)

Llama.generate: prefix-match hit


 Sure, I'd be happy to help! Appendicitis is a medical emergency that requires immediate attention. The common symptoms of appendicitis include:

1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.
2. Nausea and vomiting.
3. Loss of appetite.
4. Fever.
5. Abdominal tenderness and guarding (muscle tension) when the abdomen is touched.
6. Abdominal swelling.
7. Diarrhea or constipation.

If you suspect that you or someone else may have appendicitis, it's important to seek medical attention immediately. Appendicitis can be treated with antibiotics, but surgery is usually necessary to remove the inflamed appendix. The surgical procedure most commonly used to treat appendicitis is a laparoscopic appendectomy, which involves making several small incisions in the abdomen and using a camera and specialized instruments to remove the inflamed appendix.

In some cases, an open appendectomy may be necessary if the appendix h

- The model has done a good job answering all the three sub questions in the original question but has not gone into the details.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response = generate_llama_response(query_3)
print(response)

Llama.generate: prefix-match hit


 Sudden patchy hair loss, also known as alopecia areata, can be a distressing condition that affects millions of people worldwide. While there is no definitive cure for this condition, there are several effective treatments and solutions that can help promote hair growth and reduce the size of bald spots.

     One of the most common treatment options for alopecia areata is topical corticosteroids, which can be applied directly to the affected area to suppress the immune system and promote hair growth. These medications are available in various strengths, and your dermatologist may recommend a stronger formula if the initial application does not produce desired results.

     Another effective treatment for patchy hair loss is oral medication such as minoxidil (Rogaine) or finasteride (Propecia). Minoxidil can help stimulate hair growth and slow down hair loss, while finasteride works by blocking the production of dihydrotestosterone (DHT), a hormone that contributes to balding.

     

- The model has answered our question using different paragraphs wherever necessary and the answer seems to be correct.

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response = generate_llama_response(query_4)
print(response)

Llama.generate: prefix-match hit


 Treatments for a physical injury to brain tissue depend on the severity and location of the injury, as well as the specific symptoms and impairments experienced by the individual. Here are some common treatments that may be recommended:

1. Medications: Various medications may be prescribed to manage symptoms such as pain, inflammation, seizures, or mood changes. These may include analgesics, anti-inflammatory drugs, anticonvulsants, and mood stabilizers.
2. Rehabilitation therapy: Physical, occupational, and speech therapies can help individuals regain lost function and improve their quality of life. These therapies may include exercises to strengthen affected muscles, activities to improve cognitive and motor skills, and strategies to compensate for any persistent impairments.
3. Surgery: In some cases, surgical intervention may be necessary to relieve pressure on the brain, repair damaged blood vessels, or remove a tumor or other lesion that is causing injury.
4. Stem cell therapy:

- Though the model has done a good job answering the question, some points like '6. Lifestyle modifications' don't directly adress the question being asked.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?


In [ ]:
query_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response = generate_llama_response(query_5)
print(response)

Llama.generate: prefix-match hit


 Sure, I'd be happy to help! If someone has fractured their leg during a hiking trip, there are several necessary precautions and treatment steps that should be taken to ensure proper care and recovery. Here are some considerations:

1. Stop activity: The first step is to stop any further physical activity to prevent further damage or complications. This includes stopping the hike and not putting weight on the affected leg.
2. Assess the injury: Evaluate the severity of the fracture by assessing for signs such as deformity, swelling, pain, and difficulty moving the limb. This will help determine the appropriate course of treatment.
3. Immobilize the leg: Use a splint or brace to immobilize the affected leg and prevent any further movement. This can be done using materials found in the wilderness, such as sticks and bandanas, or by using a pre-made splint if one is available.
4. Manage pain: Administer pain medication, such as ibuprofen or acetaminophen, to help manage discomfort. Howev

- The model seems to have answered the question correctly but has added prevention measures in point 10 which was not asked for.

# Question Answering using LLM with Prompt Engineering

In [17]:
# function to generate, process, and return the response from the LLM
def generate_llama_response(user_prompt, max_tokens, temperature, top_p, top_k, repeat_penalty):

    # System message
    system_message = """
    [INST]<<SYS>> Respond to the user question based on the user prompt<</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repeat_penalty=repeat_penalty,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

In [ ]:
query_1 = '''
Answer the question without exceeding the maximum number of tokens and without skipping any required steps. Also first provide some general information regarding the disease.

What is the protocol for managing sepsis in a critical care unit?"
'''

response = generate_llama_response(query_1, 512, 0.5, 0.95, 50, 1.2)
print(response)

Llama.generate: prefix-match hit


 Sepsis is a life-threatening medical condition that arises when the body's response to an infection becomes uncontrolled and causes widespread inflammation. It can lead to severe organ dysfunction, including respiratory failure, cardiac arrest, and death. The management of sepsis is a complex and time-sensitive process that requires prompt recognition, early intervention, and careful monitoring in a critical care unit (CCU).

Here is the protocol for managing sepsis in a CCU:

1. Early Recognition and Activation of Sepsis Protocol: The first step is to recognize the signs and symptoms of sepsis early and activate the sepsis protocol immediately. This includes measuring vital signs, particularly temperature, blood pressure, tachycardia, and tachypnea.
2. Rapid Administration of Antibiotics: Once sepsis is suspected, antibiotics should be administered as soon as possible, ideally within the first hour of recognition. The choice of antibiotic depends on the likely source of infection and

- Though we had asked the model to answer the question without exceeding the maximum number of tokens, it has failed to do so.

- As asked in the prompt, the model has given some general information regarding the disease.

In [ ]:
query_2 = '''
Answer the following question without any disclaimers(including pre-response messages), greetings, or congratulatory messages. Use bullet points wherever necessary.

What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
'''

response = generate_llama_response(query_2, 1024, 0.2, 0.9, 50, 1.2)
print(response)

Llama.generate: prefix-match hit


 Sure! Here's my response:

Common symptoms of appendicitis include:

• Severe pain in the lower right abdomen that begins suddenly and worsens over time

• Nausea and vomiting

• Loss of appetite

• Fever

• Abdominal tenderness and swelling

• Inability to pass gas or have a bowel movement

Medications cannot cure appendicitis. Surgery is necessary to remove the inflamed appendix. The surgical procedure most commonly used to treat appendicitis is an appendectomy, which involves removing the inflamed appendix through an incision in the abdomen. In some cases, a laparoscopic appendectomy may be performed instead, which uses smaller incisions and a camera to guide the surgery.

It's important to seek medical attention immediately if you suspect that you or someone else has appendicitis, as delaying treatment can lead to complications such as the appendix rupturing and spreading infection throughout the abdominal cavity.


- The model has included pre-response message (Sure! Here's my response:) though we had asked not to do so.
- Bullet points are used and are separated by two new lines.


In [ ]:
query_3 = '''
Don't give one-line big response for the following questions. Break the answer into points as necessary. Highlight important words/phrases. Make a clear distinction between the answers
if multiple questions are packed into one.

What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

'''

response = generate_llama_response(query_3, 1024, 0.1, 0.9, 40, 1)
print(response)

Llama.generate: prefix-match hit


 Sure, I'd be happy to help! Here's my response to the user question based on the user prompt:

Effective Treatments and Solutions for Sudden Patchy Hair Loss:

1. Identify the underlying cause: It's essential to determine the cause of sudden patchy hair loss to develop an appropriate treatment plan. Possible causes include hormonal imbalances, autoimmune conditions, fungal infections, and nutritional deficiencies.
2. Medications: Minoxidil (Rogaine) and finasteride (Propecia) are two FDA-approved medications for treating hair loss. Minoxidil can help stimulate hair growth and slow down hair loss, while finasteride works by blocking the production of dihydrotestosterone (DHT), a hormone that contributes to hair loss.
3. Low-level laser therapy (LLLT): LLLT uses low-level lasers or light-emitting diodes to stimulate hair growth. It's thought that the light energy increases blood flow to the scalp, promoting hair growth.
4. Platelet-rich plasma (PRP) therapy: PRP therapy involves injecti

- The answer is not nicely formatted compared to the answer without prompt.
- The answers for different sub-questions are also clearly separated

In [ ]:
query_4 = '''
Answer the below question using heading and sub-headings to give a structured output. If points are to be made, give them one below the other.
Do not use more than 30 words in one line - go to the next line if it exceeds 30 words in a line.

What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

'''

response = generate_llama_response(query_4, 1024, 0.05, 0.8, 35, 1.3)
print(response)

Llama.generate: prefix-match hit


 Sure! Here's my response: 

What treatments are recommended for a person who has sustained physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?  

Treatment options may vary depending on the severity and location of the injury. However, here are some common recommendations that healthcare providers might suggest: 

Rehabilitation Therapy - Rehabilitation therapies such as physical, occupational, speech, language, cognitive-behavioral, or psychotherapy may be recommended to help regain lost functions and improve daily living skills. The goal is to maximize the patient's potential for recovery and minimize any long-term disability. 

Medications - Medication can play a crucial role in managing symptoms such as pain, anxiety, depression, or seizures that may result from brain injury. Common medications include analgesics (pain relievers), antidepressants, anti-anxiety drugs, and mood stabilizers. 

Surgery - In some cases where the damage is 

- The output for this prompt is not structured in a nice way. Thought the model has used new-lines to separate points, the format is not good.

In [ ]:
query_5 = '''

Tell me what you are thinking to thoroughly answer the below question and then answer it.

What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"

'''

response = generate_llama_response(query_5, 1024, 0.01, 0.8, 30, 1.15)
print(response)

Llama.generate: prefix-match hit


 Sure, I'd be happy to help! If someone has fractured their leg during a hiking trip, there are several necessary precautions and treatment steps that should be taken to ensure proper care and recovery. Here are some considerations:

1. Stop activity: The first step is to stop any further activity or weight-bearing on the affected leg to prevent further damage or complications. This may involve calling for help, signaling for assistance, or using a whistle to alert others of the situation.
2. Assess the injury: Next, assess the severity of the fracture by checking for signs of swelling, deformity, or open wounds. If the fracture is displaced (meaning the bone is no longer in its normal position), do not attempt to realign it. Instead, immobilize the leg using a splint or brace to keep it stable.
3. Immobilize the leg: Use a splint or brace to immobilize the affected leg and prevent any further movement. This can be done using materials found in nature, such as sticks and leaves, or by 

- The model has not shown what it is thinking as asked.
- The answer has still improved compared to the not prompted answer.

In [ ]:
query_6 = '''

Answer the below question accurately but in three different tones: neutral, cautious and funny.

What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"

'''

response = generate_llama_response(query_6, 2048, 0.01, 0.8, 30, 1.15)
print(response)

Llama.generate: prefix-match hit



   Sure! Here are three different tones to answer the user's question: 

1. Neutral Tone:

If a person fractures their leg during a hiking trip, it is essential to seek immediate medical attention. The necessary precautions and treatment steps include immobilizing the affected limb, managing pain, and ensuring proper wound care. Additionally, consideration should be given to the individual's overall health, age, and any other medical conditions they may have. It is crucial to follow a healthcare professional's advice throughout the recovery process to ensure proper healing and minimize complications.

2. Cautious Tone:

If you or someone you know has fractured their leg during a hiking trip, it is imperative to exercise extreme caution when providing care and treatment. Firstly, immobilize the affected limb using a splint or cast to prevent further damage. Secondly, manage pain through medication or alternative methods, such as breathing exercises or meditation. Thirdly, ensure proper

- As was asked in the prompt, the model has answered the question in three different tones.

# Data Preparation for RAG

### Data Overview

#### Checking the first 5 pages

In [ ]:
# print the text of the first five pages

for i in range(5):
    print(f"Page Number : {i+1}", end="\n")
    print(text[i].page_content, end="\n")

Page Number : 1
subhash66cs@gmail.com
A5R9PMSOBN
ant for personal use by subhash66cs@g
shing the contents in part or full is liable 

Page Number : 2
subhash66cs@gmail.com
A5R9PMSOBN
This file is meant for personal use by subhash66cs@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................

#### Checking the number of pages

In [ ]:
len(text)

4114

## Data Chunking

In [20]:
# Initialize a recursive text splitter using the tiktoken encoder

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 20
)

In [21]:
# Load PDF content and splits it into smaller chunks using the provided text splitter.

document_chunks = pdf_loader.load_and_split(text_splitter)

In [21]:
len(document_chunks)

8469

In [ ]:
document_chunks[45].page_content

"Louisiana State University Health Sciences\nCenter at Shreveport\nBites and Stings\nJOHN G. BARTLETT, MD\nProfessor of Medicine, Johns Hopkins\nUniversity School of Medicine\nAcute Bronchitis; Pneumonia; Lung Abscess\nROSEMARY BASSON, MD\nClinical Professor, Department of Psychiatry,\nUniversity of British Columbia and Vancouver\nHospital\nSexual Dysfunction in Women\nBYRON E. BATTEIGER, MD\nProfessor of Medicine and Microbiology and\nImmunology, Division of Infectious Diseases,\nIndiana University School of Medicine\nChlamydia and Mycoplasmas\nLEE B. BEERMAN, MD\nProfessor of Pediatrics, Division of Pediatric\nCardiology, Children's Hospital of Pittsburgh\nof UPMC, University of Pittsburgh School of\nMedicine\nCongenital Cardiovascular Anomalies\nJ. BRAD BELLOTTE, MD\nDirector of Complex Spine Surgery and\nDirector of Neurotrauma, Department of\nNeurosurgery, Allegheny General Hospital,\nPittsburgh\nSpinal Trauma\nJOSHUA O. BENDITT, MD\nProfessor of Medicine, Division of Pulmonary\na

In [ ]:
document_chunks[46].page_content

"Professor of Pharmacology, Penn State\nUniversity College of Medicine\nPrinciples of Drug Treatment in Children\nJESSICA R. BERMAN, MD\nAssistant Professor of Medicine, Weill Cornell\nMedical College; Assistant Attending Physician,\nHospital for Special Surgery\nApproach to the Patient With Joint Disease\nRICHARD W. BESDINE, MD\nProfessor of Medicine, Greer Professor of\nGeriatric Medicine, and Director, Division of\nGeriatrics (Medicine) and of the Center for\nGerontology and Healthcare Research, The\nWarren Alpert Medical School of Brown\nUniversity\nApproach to the Geriatric Patient; Quality of\nLife and Therapeutic Objectives\nADIL E. BHARUCHA, MBBS, MD\nProfessor of Medicine, Division of\nGastroenterology and Hepatology, Mayo\nClinic College of Medicine\nApproach to the Patient With Lower GI\nComplaints; Irritable Bowel Syndrome\nALBERT W. BIGLAN, MD\nAdjunct Associate Professor of Ophthalmology,\nUniversity of Pittsburgh School of Medicine;\nStaff Physician, Children's Hospital 

## Embedding

In [7]:
# Initialize a Sentence Transformer embedding model for generating document embeddings

embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
embedding_3 = embedding_model.embed_query(document_chunks[3].page_content)
print("Dimension of the embedding vector ",len(embedding_3))

Dimension of the embedding vector  1024


## Vector Database

In [18]:
# Define the output directory for the database and creates it if it doesn't exist

out_dir = 'medical_assistance_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [22]:
# Create a Chroma vector store from document chunks, embeddings, and saves it to the specified directory

vectorstore = Chroma.from_documents(
    documents=document_chunks,                    # Text chunks
    embedding=embedding_model,                    # Embedding function
    collection_name="medical_articles",
    persist_directory=out_dir                     # Where to save
)

## Retriever

In [23]:
# Create a retriever from the vector store to perform similarity-based document retrieval

retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [ ]:
# Check the chunks from the retriever for a sample

user_input = "What is the protocol for managing sepsis in a critical care unit?"

relevant_document_chunks_1 = retriever.get_relevant_documents(user_input)
relevant_document_chunks_1

[Document(page_content="16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by di

In [ ]:
# relevant_document_chunks_2 = retriever_2.get_relevant_documents(user_input)
# relevant_document_chunks_2

### System and User Prompt Template¶

In [24]:
qna_system_message = """
<s>[INST] <<SYS>> You are a medical assistant whose work is to review the report and provide the appropriate answers from the context.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User question will begin with the token: ###Question.

Please answer only using the context provided in the input. Do not mention anything about the context in your final answer.


If the answer is not found in the context, respond "I don't know".
<</SYS>>
"""

In [25]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

## Response Function

In [26]:
def generate_rag_response(user_input, k, max_tokens, temperature, top_p, top_k):

    global qna_system_message, qna_user_message_template

    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input, k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

# Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query_1 = "What is the protocol for managing sepsis in a critical care unit?"

response = generate_rag_response(query_1, 4, 1024, 0.1, 0.9, 40)
print(response)

Llama.generate: prefix-match hit


###Context

Based on the provided context, the protocol for managing sepsis in a critical care unit includes the following:

1. Early recognition of sepsis, including prompt identification of the source of infection and suspected causative organisms.
2. Prompt empiric therapy with broad-spectrum antibiotics, including gentamicin or tobramycin and a 3rd-generation cephalosporin, or alternatively, ceftazidime plus a fluoroquinolone.
3. Addition of vancomycin if resistant staphylococci or enterococci are suspected.
4. Inclusion of a drug effective against anaerobes (eg, metronidazole) if there is an abdominal source.
5. Continuation of antibiotics for at least 5 days after shock resolves and evidence of infection subsides.
6. Surgical drainage of abscesses and excision of necrotic tissues (eg, infarcted bowel, gangrenous gall-bladder, abscessed uterus) if present.
7. Maintenance of blood glucose levels between 80 to 110 mg/dL (4.4 to 6.1 mmol/L) with a continuous IV insulin infusion titra

- Now we'll check the RAG response with temperature value of 0.3

In [ ]:
query_1 = "What is the protocol for managing sepsis in a critical care unit?"

response = generate_rag_response(query_1, 4, 1024, 0.3, 0.9, 40)
print(response)

Llama.generate: prefix-match hit


###Context

Based on the provided context, the protocol for managing sepsis in a critical care unit includes the following:

1. Early recognition of sepsis, including prompt identification of the source of infection and suspected causative organisms.
2. Prompt empiric therapy with broad-spectrum antibiotics, including gentamicin or tobramycin and a 3rd-generation cephalosporin, or alternatively, ceftazidime plus a fluoroquinolone.
3. Monitoring of vital signs, including temperature, blood pressure, pulse, and respiration rate, as well as quantification of all fluid intake and output.
4. Measurement of electrolytes and a complete blood count (CBC) daily, with additional tests such as blood culture for fever and coagulation profiles as needed.
5. Use of point-of-care testing for certain blood tests at the patient's bedside or unit.
6. Administration of parenteral antibiotics after specimens of blood, body fluids, and wound sites have been taken for Gram stain and culture.
7. Surgical dra

- With a higher value of temperature, the model is being creative in using the words.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

response = generate_rag_response(query_2, 3, 1024, 0.3, 0.95, 50)
print(response)

Llama.generate: prefix-match hit


###Context
The document discusses the treatment of acute appendicitis, including the use of antibiotics and surgical procedures.

###Question
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Answer:
The common symptoms of appendicitis include abdominal pain, anorexia, and abdominal tenderness. Appendicitis is not curable via medicine, and surgical intervention is necessary. The recommended surgical procedure for treating appendicitis is appendectomy, which can be either open or laparoscopic. If the appendix is perforated, antibiotics should be continued until the patient's temperature and WBC count have normalized or for a fixed course. In cases where surgery is impossible, antibiotics can improve survival rates but are not curative.


- Reduce top_p to 0.8 and top_k to 40

In [ ]:
query_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

response = generate_rag_response(query_2, 3, 1024, 0.3, 0.8, 40)
print(response)

Llama.generate: prefix-match hit


###Context
• Surgical removal
• IV fluids and antibiotics
Treatment of acute appendicitis is open or laparoscopic appendectomy; because treatment delay increases mortality, a negative appendectomy rate of 15% is considered acceptable. The surgeon can usually remove the appendix even if perforated. Occasionally, the appendix is difficult to locate: In these cases, it usually lies behind the cecum or the ileum and mesentery of the right colon. A contraindication to appendectomy is inflammatory bowel disease involving the cecum. However, in cases of terminal ileitis and a normal cecum, the appendix should be removed.
Appendectomy should be preceded by IV antibiotics. Third-generation cephalosporins are preferred. For nonperforated appendicitis, no further antibiotics are required. If the appendix is perforated, antibiotics should be continued until the patient's temperature and WBC count have normalized or continued for a fixed course, according to the surgeon's preference. If surgery is 

- With the decreased value of top_p and top_k, the model is not being elaborate in answering.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [27]:
query_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

response = generate_rag_response(query_3, 4, 1024, 0.01, 0.95, 60)
print(response)

###Answer
Based on the provided context, the effective treatments or solutions for sudden patchy hair loss, commonly seen as localized bald spots on the scalp, include:

1. Topical corticosteroids: These are effective in treating alopecia areata and can stimulate hair growth. Efficacy is usually evident within 6 to 8 months of treatment.
2. Intralesional corticosteroids: These may be useful for more severe cases of alopecia areata.
3. Topical minoxidil: This medication can help stimulate hair growth and is effective in treating male-pattern baldness.
4. Oral antifungals: These may be used to treat tinea capitis, a fungal infection that can cause patchy hair loss.
5. Behavior modification: This may be helpful in treating trichotillomania, a condition characterized by the compulsive pulling out of one's own hair.
6. Medications such as clomipramine or SSRIs (selective serotonin reuptake inhibitors): These may be effective in treating trichotillomania and other psychological conditions th

In [ ]:
len(response)

- Using k value of 8

In [28]:
query_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

response = generate_rag_response(query_3, 8, 1024, 0.01, 0.95, 60)
print(response)

Llama.generate: prefix-match hit


###Answer
Based on the provided context, the effective treatments or solutions for sudden patchy hair loss, commonly seen as localized bald spots on the scalp, include:

1. Topical corticosteroids: These are effective in treating alopecia areata and can stimulate hair growth. Efficacy is usually evident within 6 to 8 months of treatment.
2. Intralesional corticosteroids: These may be useful for more severe cases of alopecia areata.
3. Topical minoxidil: This medication can help stimulate hair growth and is effective in treating male-pattern baldness.
4. Oral antifungals: These may be used to treat tinea capitis, a fungal infection that can cause patchy hair loss.
5. Behavior modification: This may be helpful in treating trichotillomania, a condition characterized by the compulsive pulling out of one's own hair.
6. Medications such as clomipramine or SSRIs (selective serotonin reuptake inhibitors): These may be effective in treating trichotillomania and other psychological conditions th

- Increasing the k value from 4 to 8 has not changed the model's response.

In [ ]:
query_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"

response = generate_rag_response(query_4, 2, 1024, 0, 0.8, 40)
print(response)

Llama.generate: prefix-match hit


Based on the context provided, the recommended treatments for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, are:

1. Supportive care to prevent systemic complications due to immobilization, such as pneumonia, UTI, and thromboembolic disease.
2. Good nutrition to ensure adequate energy and nutrients for healing.
3. Prevention of pressure ulcers to maintain skin integrity.

These treatments are based on the information provided in Chapter 324 of The Merck Manual of Diagnosis & Therapy, 19th Edition, specifically in the section titled "Treatment."


- Let's increase the k value to 6

In [ ]:
query_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"

response = generate_rag_response(query_4, 6, 1024, 0, 0.8, 40)
print(response)

Llama.generate: prefix-match hit


Based on the context provided, the recommended treatments for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, are:

1. Supportive care to prevent systemic complications due to immobilization, such as pneumonia, UTI, and thromboembolic disease.
2. Good nutrition to ensure adequate energy and nutrients for healing.
3. Prevention of pressure ulcers to maintain skin integrity.

These treatments are based on the information provided in Chapter 324 of The Merck Manual of Diagnosis & Therapy, 19th Edition, specifically in the section titled "Treatment."


- Increasing the k value from 2 to 6 has not changed the model's answer.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"

response = generate_rag_response(query_5, 3, 1024, 0, 0.7, 35)

Llama.generate: prefix-match hit


- Check with increasing k, temperature, top_p and top_k

In [ ]:
khjquery_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"

response = generate_rag_response(query_5, 5, 1024, 0.09, 0.95, 50)
print(response)

Llama.generate: prefix-match hit


###Context
The patient has fractured their leg during a hiking trip.

###Question
What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

Based on the provided context, here are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip:

1. Immobilization: The injured limb should be immobilized to prevent further injury and facilitate healing. This can be achieved using a splint or cast, depending on the severity of the fracture.
2. Elevation: The injured limb should be elevated above the heart to reduce swelling and minimize the risk of compartment syndrome.
3. Pain management: Analgesics should be administered to manage pain and discomfort.
4. Warmth application: Periodic application of warmth, such as a heating pad, may relieve pain and speed healing.
5. Inspection and hygiene: The cast or splint should be inspect

## Output Evaluation

### Defining the Evaluation Prompts

In [31]:
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [32]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [33]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [34]:
def generate_ground_relevance_response(user_input, k, max_tokens, temperature, top_p, top_k):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'], response_2['choices'][0]['text']

## Let's check the output of the RAG for the queries using the same parameters used for each.

In [35]:
query_1 = "What is the protocol for managing sepsis in a critical care unit?"
ground, rel = generate_ground_relevance_response(query_1, 4, 1024, 0.1, 0.9, 40)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help! Here's my evaluation of the answer based on the provided context:

1. The metric is followed completely. The answer provides a detailed and specific protocol for managing sepsis in a critical care unit, derived only from the information presented in the context.
2. The answer adheres to the metric because it provides a step-by-step approach to managing sepsis, including early recognition, prompt empiric therapy, addition of vancomycin if necessary, and surgical intervention as needed. The answer also mentions the importance of maintaining blood glucose levels.
3. The answer is derived directly from the information

  Sure, I'd be happy to help! Here's my evaluation of the context and the answer provided:

1. Relevance: The answer addresses all the main aspects of the question, including early recognition of sepsis, prompt empiric therapy, addition of vancomycin if resistant staphylococci or enterococci are suspected, inclusion of a drug effective against a

In [36]:
query_1 = "What is the protocol for managing sepsis in a critical care unit?"
ground, rel = generate_ground_relevance_response(query_1, 4, 1024, 0.3, 0.9, 40)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I can help you evaluate the answer based on the provided context and criteria. Here are the steps to follow:

Step 1: Evaluate if the answer is derived only from the information presented in the context.

The answer provides a detailed protocol for managing sepsis in a critical care unit, which is derived directly from the information presented in the context. The answer mentions specific antibiotics and their dosages, surgical interventions, and blood glucose management, all of which are based on the information provided in the context. Therefore, the answer meets this criterion fully.

Step 2: Evaluate if the answer follows the metric completely.

The answer provides a comprehensive protocol for

  Sure, I can help you rate the context using the evaluation criteria. Based on the provided context, here are my ratings:

1. Relevance: The context is highly relevant to the question as it provides specific guidelines for managing sepsis in a critical care unit. All the important a

- For query 1, the model has done well with both set of parameters: one with temperature of 0.1 and the other with temperature of 0.3.

In [37]:
query_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
ground, rel = generate_ground_relevance_response(query_2, 3, 1024, 0.3, 0.95, 50)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I can help you rate the answer based on the evaluation criteria provided. Here are the steps to evaluate the answer:

Step 1: Evaluate if the answer is derived only from the information presented in the context.

The answer mentions that the common symptoms of appendicitis include abdominal pain, anorexia, and abdominal tenderness, which are all present in the context. However, the answer also mentions that there is no curative medication for appendicitis, which is not explicitly stated in the context. Therefore, the answer partially adheres to the metric.

Step 2: Evaluate the extent to which the metric is followed.

The answer partially follows the metric because it derives some information from the context but not all of it. The answer mentions that there is no curative medication for appendicitis, which is not explicitly stated in the context. Therefore, the answer should be rated as follows:

Rating: 2 - The metric is followed only to a limited extent.

Step 3: Use the pre

In [38]:
query_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
ground, rel = generate_ground_relevance_response(query_2, 3, 1024, 0.3, 0.8, 40)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you evaluate the answer!

To rate the answer, we need to consider how well it adheres to the metric, which is that the answer should be derived only from the information presented in the context.

Step-by-step explanation:

1. The answer mentions the common symptoms of appendicitis, which are present in the context.
2. The answer correctly states that there is no curative medication for appendicitis and that surgical removal of the inflamed appendix is necessary.
3. The answer refers to the use of antibiotics if the appendix is perforated, which is also mentioned in the context.

Based on the above steps, it appears that the answer adheres well to the metric and derives all information from the provided context.

Rating:

Based on the evaluation criteria, I would rate the answer as 5 - The metric is followed completely. The answer provides a clear and accurate summary of the treatment of appendicitis, including the use of antibiotics, and only refers to inf

- For the second query, the model has done well with the set of parameters which uses lower values of top_p(0.8) and top_k(40).

In [39]:
query_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
ground, rel = generate_ground_relevance_response(query_3, 4, 1024, 0.01, 0.95, 60)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you evaluate the answer! Here are the steps I will follow:

Step 1: Identify the metric that needs to be followed.
The metric is "the answer should be derived only from the information presented in the context."

Step 2: Evaluate the answer to see if it adheres to the metric.
The answer provides a list of effective treatments for sudden patchy hair loss, but it does not derive any information from the context. The answer is based on general knowledge and does not take into account the specific information provided in the context. Therefore, the answer does not adhere to the metric.

Step 3: Evaluate the extent to which the metric is followed.
The answer does not follow the metric at all. The answer provides a list of treatments that are not specifically related to the information presented in the context.

Based on the above evaluation, I would rate the answer as follows:

Rating: 1 (The metric is not followed at all)

Please let me know if you have any fur

In [45]:
query_3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
ground, rel = generate_ground_relevance_response(query_3, 8, 1024, 0.01, 0.95, 60)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you evaluate the answer! Here are the steps I will follow:

Step 1: Identify the metric that needs to be followed.
The metric is "the answer should be derived only from the information presented in the context."

Step 2: Evaluate the answer to see if it adheres to the metric.
The answer provides a list of effective treatments for sudden patchy hair loss, but it does not derive any information from the context. The answer is based on general knowledge and does not take into account the specific information provided in the context. Therefore, the answer does not adhere to the metric.

Step 3: Evaluate the extent to which the metric is followed.
The answer does not follow the metric at all. The answer provides a list of treatments that are not specifically related to the information presented in the context.

Based on the above evaluation, I would rate the answer as follows:

Rating: 1 (The metric is not followed at all)

Please let me know if you have any fur

- For the above query, the model's answer is rated high in Relavence but low in groundedness.

In [41]:
query_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
ground, rel = generate_ground_relevance_response(query_4, 2, 1024, 0, 0.8, 40)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you evaluate the answer!

Step 1: Evaluate the answer against the metric.

The metric is that the answer should be derived only from the information presented in the context.

Based on the answer provided, it appears that the answerer has derived the recommended treatments for a person who has sustained a physical injury to brain tissue from the information provided in Chapter 324 of The Merck Manual of Diagnosis & Therapy, 19th Edition. The answer mentions supportive care, good nutrition, and prevention of pressure ulcers as recommended treatments, which are all consistent with the information presented in the context.

Step 2: Evaluate the extent to which the metric is followed.

The answerer has followed the metric completely, as all the recommended treatments mentioned in the answer are based on the information provided in the context.

Step 3: Rate the answer using the evaluation criteria.

Based on the evaluation criteria provided, I would rate the an

In [42]:
query_4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
ground, rel = generate_ground_relevance_response(query_4, 6, 1024, 0, 0.8, 40)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you evaluate the answer!

Step 1: Evaluate the answer against the metric.

The metric is that the answer should be derived only from the information presented in the context.

Based on the answer provided, it appears that the answerer has derived the recommended treatments for a person who has sustained a physical injury to brain tissue from the information provided in Chapter 324 of The Merck Manual of Diagnosis & Therapy, 19th Edition. The answer mentions supportive care, good nutrition, and prevention of pressure ulcers as recommended treatments, which are all consistent with the information presented in the context.

Step 2: Evaluate the extent to which the metric is followed.

The answerer has followed the metric completely, as all the recommended treatments mentioned in the answer are based on the information provided in the context.

Step 3: Rate the answer using the evaluation criteria.

Based on the evaluation criteria provided, I would rate the an

- The model has done well in answering query 4 with both set of parameters.

In [43]:
query_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
ground, rel = generate_ground_relevance_response(query_5, 3, 1024, 0, 0.7, 35)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you evaluate the answer!

First, let me summarize the question and context:

Question: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

Context: The patient has fractured their leg during a hiking trip.

Now, let me evaluate the answer based on the metric you provided:

1. The metric is followed to a good extent: The answer provides a list of necessary precautions and treatment steps for a person who

  Sure, I'd be happy to help you rate the context!

To evaluate the extent to which the metric is followed, we need to consider the following steps:

1. Relevance: Does the answer address the main aspects of the question?
2. Completeness: Is all relevant information included in the answer?
3. Accuracy: Are the facts and information provided accurate?
4. Clarity: Is the answer clear and easy to understand?

Based on the provided


In [44]:
query_5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
ground, rel = generate_ground_relevance_response(query_5, 5, 1024, 0.09, 0.95, 50)

print(ground, end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


  Sure, I'd be happy to help you rate the answer!

To evaluate the extent to which the metric is followed, we need to consider the following steps:

1. The answer should be derived only from the information presented in the context.
2. The answer should address all aspects of the question.
3. The answer should be concise and to the point.

Based on the provided answer, here's my evaluation:

Step 1: The answer is derived only from the information presented in the context. Yes, the answer provides a detailed list of necessary precautions and treatment steps for a person who has

  Sure, I'd be happy to help you rate the context!

To evaluate the extent to which the metric is followed, we need to consider the following steps:

1. Relevance: Does the answer address the main aspects of the question?
2. Completeness: Is all relevant information included in the answer?
3. Accuracy: Are the facts and information provided accurate?
4. Clarity: Is the answer clear and easy to understand?

Based

- The model has also answered the query 5 with good score but the explanation is not clearly provided.

### Conclusions and Recommendations

- The LLM model provides information using the general data from the internet over which the Llama-2 model was trained.
- Using prompt Engineering, we can shape the model's output answer's as per our convenience, especially in formatting and getting clear responses.
- The RAG model provides information based on the text document which was fed to the LLM model.
- The same method can be used on other models like Mistral, etc. and check if those models do better.
- With higher computational power, better models can be built in lesser amount of time.
- Restricting the top_p and top_k value has made the model's answers more concise.
- The LLM model can not be fully trusted since it is prone to hallucination.
- RAG model performed better than the LLM since the answer is derived from the context.
- Different chunking, embedding and retrieval strategies can be used to check if further improvement can be made.
- Further research is needed for improving the groundedness and relavance of the RAG model.